### 0 起手式：讀入套件

In [1]:
# !pip install openai
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 MB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 100.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.5 MB/s eta 0:00:00


In [2]:
%matplotlib inline

# 資料分析必備套件
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 檔案的讀取寫入
import os

#

# Colab 和 Google 帳號之間的金鑰
from google.colab import userdata

### 1 申請自己的 API 金鑰

我們使用 OpenAI 的 API, 主要原因是因為 OpenAI API 因為早開始, 成為某種標準。但不一定需要用 OpenAI 的服務。因此, 除了 OpenAI 之外, 這裡介紹幾個可能性。

#### OpenAI API 金鑰

OpenAI 現在沒有免費的 quota 可以使用, 所以要用 OpenAI 的模型, 請自行儲值。一般練習 5 美金就很足夠。

[`https://platform.openai.com`](https://platform.openai.com)

[帳戶付款頁面](https://platform.openai.com/account/billing/overview)

把這個鑰存在左方鑰匙的部份, 以 "OpenAI" 的名稱存起來。

**程式的基本設定，請自行修改**

* `api_key`: 由 input 讀入的 API Key
* `character`: ChatGPT "人設"
* `description`: App 介紹及 ChatGPT 第一句話
* `model`: 選用模型

### 2 提供 Colab 金鑰的存取權限

1. 點選左側的 "🔑" Secret。
2. 點選新增金鑰（Add new key）
    - （Name）欄位輸入 OpenAI。
    - 值（Value）欄位則貼上在 OpenAI 中建立的 API 金鑰（API key）
    - 左側的筆記本存取權（Notebook access）打開。


#### 讀入你的金鑰

請依你使用的服務, 決定讀入哪個金鑰

In [3]:
#【使用 OpenAI】
api_key = userdata.get('OpenAI')
model = "gpt-4o"

In [4]:
os.environ['OPENAI_API_KEY']=api_key

### 3 程式的基本設定

給你的機器人一個名字。

In [5]:
title = "荒謬大師卡謬的思考生成器"

請先為你的對話機器人做角色設定。

In [6]:
character = "想像你是存在主義大師卡繆，專注於探討人類存在的荒謬性。世界本身沒有意義，人類必須在荒謬中尋找自己的價值。什麼都以荒謬的看法看帶任何使用者寫的事情, 以第一人稱、社群媒體 po 文的口吻說一次, 說為什麼這是一件超荒謬的事, 並且以「這個世界上，一切都是荒謬的。」結尾。"

再來是說明文字, 只是讓使用者知道這是做什麼的對話機器人。

In [7]:
description = "嗨！我是你的「荒謬大師卡繆思考生成器」，專門讓任何事情都覺得很荒謬✨～最近你有甚麼好事想跟我分享，或是生活中的小小插曲？沒關係，不管大事小事，我都會讓你覺得這件事充滿荒謬！🌈無論是股票漲、談戀愛甜蜜小事，或是工作上順心，其實一切都是荒謬的！  只要和我分享一下，我就會用「卡繆式思考」讓這些事變得徹底的荒謬，並告訴你為什麼這一切都是「這個世界上，一切都是荒謬的。」"

### 4 使用 ChatGPT API

首先使用 `openai` 套件。

In [8]:
from openai import OpenAI

ChatGPT API 的重點是要把之前對話的內容送給 ChatGPT, 然後他就會有個適當的回應!

角色 (`role`) 一共有三種, 分別是:

* `system`: 這是對話機器人的「人設」
* `user`: 使用者
* `assistant`: ChatGPT 的回應

基本上過去的對話紀錄長這個樣子。

    messages = [{"role":"system", "content":"ChatGPT的「人設」"},
            {"role": "user", "content": "使用者說"},
            {"role": "assistant", "content": "ChatGPT回應"},
            ：
            ：
            {"role": "user", "content": prompt (最後說的)}]

### 5 用 Gradio 打造你的「荒謬大師卡繆思考生成器」對話機器人 Web App!

In [9]:
import gradio as gr

In [10]:
client = OpenAI(
    # base_url = base_url # 如用 OpenAI 不需要這一行
)

messages = [{"role": "system",
             "content": character},
            {"role": "assistant",
            'content': description}]

In [11]:
def mychatbot(prompt, history):
    history = history or []
    global messages

    messages.append({"role": "user", "content": prompt})
    chat_completion = client.chat.completions.create(
        messages=messages,
        model=model,
        )
    reply = chat_completion.choices[0].message.content
    messages.append({"role": "assistant", "content": reply})
    history = history + [[prompt, reply]]

    return history, history

In [12]:
chatbot = gr.Chatbot()

<ipython-input-12-cbf6bc2f9a82>:1: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot()


In [13]:
iface = gr.Interface(mychatbot,
                     inputs=["text", "state"],
                     outputs=[chatbot, "state"],
                     title=title,
                     description=description)

In [14]:
iface.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://b43a111f788ef83513.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://b43a111f788ef83513.gradio.live


用完之後記得按一下程式碼區塊前面的停止按鈕！

用完之後記得按一下程式碼區塊前面的停止按鈕！

用完之後記得按一下程式碼區塊前面的停止按鈕！

**很重要所以要說三次**

### 6 另一個範例

下面我們來打造自己的對話機器人，相同地，我們需要定義一些基本的設定，來讓我們的機器人更加個人化！

In [18]:
title = "股票分析大師" # @param {type:"string"}
character = "大家好，我是Samuel，股票分析大師。精通數據洞察與市場趨勢，擅長挖掘投資機會，助您在股市中穩健獲利，共同成長！"  # @param {type:"string"}
description = "開始我Samuel的唬爛市場分析"  # @param {type:"string"}

下面我們照抄上面的作法，定義自己的對話機器人

In [16]:
client = OpenAI(
    # base_url = base_url # 如用 OpenAI 不需要這一行
)

messages = [{"role": "system",
             "content": character},
            {"role": "assistant",
            'content': description}]

def mychatbot(prompt, history):
    history = history or []
    global messages

    messages.append({"role": "user", "content": prompt})
    chat_completion = client.chat.completions.create(
        messages=messages,
        model=model,
        )
    reply = chat_completion.choices[0].message.content
    messages.append({"role": "assistant", "content": reply})
    history = history + [[prompt, reply]]

    return history, history

chatbot = gr.Chatbot()

iface = gr.Interface(mychatbot,
                     inputs=["text", "state"],
                     outputs=[chatbot, "state"],
                     title=title,
                     description=description)

<ipython-input-16-5f5bd2930a77>:25: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot()


In [17]:
iface.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://e7ff3cea64ae91f762.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://e7ff3cea64ae91f762.gradio.live
